In [4]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import os
from io import StringIO


In [5]:
# see a few json files 
arr = os.listdir('retroData/')
arr = [i for i in arr if i.endswith('json')]
arr[0:5]

['IBV_116891.json',
 'IBV_65258.json',
 'IBV_44155.json',
 'IBV_2152.json',
 'IBV_48875.json']

### Transfer the py file here in the notebook chunk (current version is still Anson's notebook#2)

In [6]:
from datetime import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import joblib
import json
import sys
import logging
def ibvmodel(jsonstr, modelfilepath):
    
    logging.basicConfig(filename="IBVModel_WarningErrorLog_2023.log", level=logging.WARNING, 
            format="%(asctime)s - %(levelname)s - %(message)s")
    stepname = '1 Loading Json'

    try:
        # Load and Read the JSON file
        json_temp = json.loads(jsonstr)
        # Load Json file's historical transaction part
        df_ibv = pd.read_json(StringIO(json.dumps(json_temp['Historical_Transactions'])))

        # Data Cleaning
        stepname = '2 Data Cleaning'
        df_ibv['IBV_Credit'] = np.where(df_ibv['amount']<0, -1*df_ibv['amount'],0)
        df_ibv['IBV_Debit'] = np.where(df_ibv['amount']>=0, df_ibv['amount'],0)
        df_ibv = df_ibv.rename(columns={'date':'IBV_Date','category':'IBV_BalCategory','name':'IBV_Description'})
        df_ibv['IBV_Date'] = pd.to_datetime(df_ibv['IBV_Date'])
        df_ibv['Year'] = df_ibv['IBV_Date'].apply(lambda x: x.year)
        df_ibv['Month'] = df_ibv['IBV_Date'].apply(lambda x: x.month)
        df_ibv = df_ibv[['account_id','IBV_Credit','IBV_Debit','IBV_Date','Year','Month','IBV_BalCategory','IBV_Description']]
        df_ibv['isLoan'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Service","Financial","Loans and Mortgages"],
                                            ["Travel and Transportation","Auto Loan"], ["Education","Student Loan"],
                                            ["Miscellaneous","Point of Sale Loan"],["Miscellaneous","Insolvency Loan"],
                                            ["Miscellaneous","Other Loan"],["Miscellaneous","Micro Loan"]] else 0)
        df_ibv['regularPayroll'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if x in [["Transfer","Payroll"], ["Transfer","Payroll",'Benefits'],
                                    ["Income","Pension"],["Income","Government"],["Income","Wages and Salary"]] else 0)
        df_ibv['isFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","ATM"],["Bank Fees","Cash Advance"],
                                                ["Bank Fees","Excess Activity"], ["Bank Fees","Foreign Transaction"],
                                                ["Bank Fees","Fraud Dispute"],["Bank Fees","Insufficient Funds"],
                                                ["Bank Fees","Late Payment"], ["Bank Fees","Overdraft"],
                                                ["Bank Fees","Wire Transfer"],["Bank Fees"],
                                                ["Fees and Charges","ATM Fees"], ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isNSFFees'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Bank Fees","Insufficient Funds"],["Bank Fees","Overdraft"],
                                            ["Fees and Charges","Service Fees"]]) else 0)
        df_ibv['isDeposit'] = df_ibv.IBV_BalCategory.apply(lambda x: 1 if (x in [["Transfer","Deposit","ATM"],
                                                ["Transfer","Deposit","Check"],["Transfer","Deposit"], ["Miscellaneous Income","Deposit"]]) else 0)
        df_ibv['IBV_BalCategory'] = df_ibv['IBV_BalCategory'].apply(str)
        accountid = df_ibv['account_id'].values[0]

        # Load Json file's historical balance part
        primary_account_info = [i for i in json_temp['Historical_Balance']['report']['items'][0]['accounts'] 
                            if i['account_id']==df_ibv.account_id.values[0]]
        if primary_account_info == []:
            df_balance = pd.DataFrame( [[0.00, '2023-02-06', 'CAD' , None]], columns = ['IBV_Balance','date',
                                                        'iso_currency_code', 'unofficial_currency_code'] )
        else:
            df_balance = pd.DataFrame(primary_account_info[0]['historical_balances'])
            df_balance = df_balance.rename(columns={'current':'IBV_Balance'})
        df_balance.date = pd.to_datetime(df_balance.date)
            
        curdate = date.today()
        curdate = df_balance['date'].max().date() # remove it before deployment
        
        df_balance['within_last_7day'] = df_balance.apply(lambda x: (curdate - x['date'].date()).days<=7, axis=1)

        ## Step 2: Feature Generation for IBV: to get the input of IBV_Model

        # Feature Generation

        # # Max History
        stepname = '3 Feature Generation'
        
        f1 = pd.DataFrame([[df_ibv.account_id.values[0],(df_ibv['IBV_Date'].max() - df_ibv['IBV_Date'].min()).days]],
                      columns = ['account_id','MaxHistory'])

        # # CREDIT_TO_DEBIT_RATIO_MEAN, CREDIT_TO_DEBIT_RATIO_SD
        f2 = (
        df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']]
        .sum(numeric_only=True)
        .reset_index()
        )
        f2['CREDIT_TO_DEBIT_RATIO'] = f2['IBV_Credit'] / f2['IBV_Debit']
        f2 = f2.groupby('account_id').agg({'CREDIT_TO_DEBIT_RATIO':['mean','std']}).reset_index()
        f2.columns = ['account_id','CREDIT_TO_DEBIT_RATIO_MEAN','CREDIT_TO_DEBIT_RATIO_SD']

        # # DAILY_DEBIT_AMOUNT_MEAN, DAILY_INCOME_MEAN
        f3 = (
        df_ibv[df_ibv.isLoan==0]
        .groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']]
        .sum(numeric_only=True)
        .reset_index()
        )
        f3['IBV_Credit'] = f3['IBV_Credit'] / 30
        f3['IBV_Debit'] = f3['IBV_Debit'] / 30
        f3 = f3.groupby('account_id').mean()[['IBV_Debit','IBV_Credit']].reset_index()
        f3.columns = ['account_id','DAILY_DEBIT_AMOUNT_MEAN','DAILY_INCOME_MEAN']

        # # DAILY_INCOME_REGULAR_MEAN
        f4 = df_ibv[df_ibv.regularPayroll==1].groupby(['account_id','Year','Month']).sum(numeric_only=True)['IBV_Credit'].reset_index()
        f4['IBV_Credit'] = f4['IBV_Credit'] / 30
        f4 = f4.groupby('account_id').mean()['IBV_Credit'].reset_index()
        f4.columns = ['account_id','DAILY_INCOME_REGULAR_MEAN']

        # # DEBIT_AMOUNT_AVG, DEBIT_AMOUNT_SD, DEBIT_COUNT_AVG,DEBIT_COUNT_SD
        f5 = df_ibv.groupby(['account_id','Year','Month']).sum(numeric_only=True)['IBV_Debit'].reset_index()
        f5 = f5.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f5.columns = ['account_id','DEBIT_AMOUNT_MEAN','DEBIT_AMOUNT_SD']

        f6 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index()
        f6 = f6.groupby('account_id').agg({'IBV_Debit':['mean','std']}).reset_index()
        f6.columns = ['account_id','DEBIT_COUNT_MEAN','DEBIT_COUNT_SD']

        # # DEBIT_AMOUNT_Z, DEBIT_COUNT_Z
        f71 = df_ibv.groupby(['account_id','Year','Month']).sum(numeric_only=True)['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f71 = f71.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f71 = f71.merge(f5,on='account_id')
        f71['DEBIT_AMOUNT_Z'] = (f71['IBV_Debit'] - f71['DEBIT_AMOUNT_MEAN'])/f71['DEBIT_AMOUNT_SD']

        f72 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id','Year','Month']).count()['IBV_Debit'].reset_index().sort_values(by=['account_id','Year','Month'],ascending=False)
        f72 = f72.groupby('account_id').first().reset_index()[['account_id','IBV_Debit']]
        f72 = f72.merge(f6,on='account_id')
        f72['DEBIT_COUNT_Z'] = (f72['IBV_Debit'] - f72['DEBIT_COUNT_MEAN'])/f72['DEBIT_COUNT_SD']

        f7 = f71[['account_id', 'DEBIT_AMOUNT_Z']].merge(f72[['account_id','DEBIT_COUNT_Z']], on='account_id', how='left')


        # # HIGHEST_PAY_DEPOSIT_MEAN, HIGHEST_PAY_FREQUENCY
        f81 = df_ibv.groupby(['account_id','IBV_BalCategory','Year','Month']).sum(numeric_only=True)['IBV_Credit'].reset_index()
        f81 = f81.groupby(['account_id','IBV_BalCategory']).mean()['IBV_Credit'].reset_index().sort_values(by=['account_id','IBV_Credit','IBV_BalCategory'],ascending=False)
        f82 = f81.groupby('account_id').first().reset_index()[['account_id','IBV_BalCategory']]
        f8 = f81.merge(f82, on =['account_id','IBV_BalCategory'], how='inner')[['account_id','IBV_Credit']]
        f8.columns = ['account_id','HIGHEST_PAY_DEPOSIT_MEAN']

        f9 = df_ibv[df_ibv.IBV_Credit>0].merge(f82, on =['account_id','IBV_BalCategory'], how='inner').groupby(['account_id','Year','Month']).count()['IBV_Credit'].reset_index()
        f9 = f9.groupby('account_id')['IBV_Credit'].mean().reset_index()[['account_id','IBV_Credit']]
        f9.columns = ['account_id','HIGHEST_PAY_FREQUENCY']

        # # INCOME_SOURCES_COUNT
        f10 = df_ibv[(df_ibv.IBV_Credit>0)&(df_ibv.IBV_BalCategory.isin(['income/investment_income','income/paycheck','income/bonus','income/government',
        'income','income/pension','income/child_support']))].groupby(['account_id','IBV_BalCategory']).size().reset_index()
        f10 = f10.groupby('account_id').size().reset_index()
        f10.columns = ['account_id','INCOME_SOURCES_COUNT']



        # # MONTH_INFLOW_MEAN, MONTH_INFLOW_SD, MONTH_OUTFLOW_MEAN, MONTH_OUTFLOW_SD
        f11 = (
        df_ibv.groupby(['account_id','Year','Month'])[['IBV_Credit','IBV_Debit']]
        .sum(numeric_only=True)
        .reset_index()
        )
        f11 = f11.groupby('account_id').agg({'IBV_Credit':['mean','std'],'IBV_Debit':['mean','std']}).reset_index()
        f11.columns = ['account_id','MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD', 'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD']


        # #MONTHS_WITH_FEES_RATE
        f12_ = df_ibv.copy()
        f12 = f12_.groupby(['account_id','Year','Month']).sum(numeric_only=True)['isFees'].reset_index()
        f12['MONTHS_WITH_FEES_RATE'] = np.where(f12['isFees']>0,1.0,0.0)
        f12 = f12.groupby('account_id').mean()['MONTHS_WITH_FEES_RATE'].reset_index()[['account_id','MONTHS_WITH_FEES_RATE']]

        # # MONTHS_WITH_EMPLOYMENT_RATE
        f12_['Emp'] = (df_ibv['IBV_BalCategory']=='income/paycheck').astype(int) #!!!!!
        f13 = f12_.groupby(['account_id','Year','Month']).sum(numeric_only=True)['Emp'].reset_index()
        f13['MONTHS_WITH_EMPLOYMENT_RATE'] = np.where(f13['Emp']>0,1.0,0.0)
        f13 = f13.groupby('account_id').mean()['MONTHS_WITH_EMPLOYMENT_RATE'].reset_index()[['account_id','MONTHS_WITH_EMPLOYMENT_RATE']]

        # # NO_ACTIVITY_RATE
        f14 = df_ibv.groupby('account_id')['IBV_Date'].nunique().reset_index()
        f14 = f14.merge(f1, on='account_id')[['account_id','IBV_Date','MaxHistory']]
        f14['NO_ACTIVITY_RATE'] = f14['IBV_Date'] / f14['MaxHistory']
        f14 = f14[['account_id','NO_ACTIVITY_RATE']]

        # # OD_AND_NSF_FEES_DAILY
        f15 = df_ibv[df_ibv.isNSFFees==1].groupby('account_id')['IBV_Debit'].sum().reset_index()
        f15 = f15.merge(f1, on='account_id')[['account_id','IBV_Debit','MaxHistory']]
        f15['OD_AND_NSF_FEES_DAILY'] = f15['IBV_Debit'] / f15['MaxHistory']
        f15 = f15[['account_id','OD_AND_NSF_FEES_DAILY']]

        # # RECURRENT_COUNT, RECURRENT_RATE
        f16 = df_ibv[df_ibv.IBV_Debit>0].groupby(['account_id', 'Year', 'Month', 'IBV_Debit', 'IBV_BalCategory']).size()
        f16 = f16.groupby(['account_id','IBV_Debit','IBV_BalCategory']).size().reset_index()
        f16.columns = ['account_id','IBV_Debit', 'IBV_BalCategory', 'CNT']
        f16 = f16[(f16['CNT']>=5)&(f16['IBV_Debit']>=10)]
        f16 = f16.groupby('account_id').agg({'IBV_BalCategory':'nunique','IBV_Debit':'sum'}).reset_index()
        f16.columns = ['account_id','RECURRENT_COUNT','RECURRING_RATE']
        
        f17 = pd.DataFrame([[accountid, df_balance['IBV_Balance'].mean(), df_balance['IBV_Balance'].std(),
                            (df_balance['IBV_Balance']>200).mean(), int(df_balance['IBV_Balance'].values[0]>200),
                            int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].values.mean())>200),
             int((df_balance[df_balance.within_last_7day==True]['IBV_Balance'].mean())>(df_balance[df_balance.within_last_7day==False]['IBV_Balance'].mean())) ]], 
              columns = ['account_id','BALANCE_MEAN','BALANCE_SD','BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                         'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D'])
        
        # # Red Zone features added by Coral: loan, payroll, deposit, total credit, total debit
        f18 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Debit']>60)].groupby('account_id').agg({'IBV_Debit':['count','sum']}).reset_index()
        f18.columns = ['account_id', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT']
        
        f19 = df_ibv[(df_ibv['isLoan'] == 1)&(df_ibv['IBV_Credit']>60)].groupby('account_id').agg({'IBV_Credit':['count','sum']}).reset_index()
        f19.columns = ['account_id', 'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT']
        
        f20 = df_ibv[(df_ibv['regularPayroll']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f20.columns = ['account_id','PAYROLL_AMOUNT']
        

        f21 = df_ibv[(df_ibv['isDeposit']==1) & (df_ibv['IBV_Credit']>50)].groupby('account_id')['IBV_Credit'].sum().reset_index()
        f21.columns = ['account_id','DEPOSIT_AMOUNT']
        f22 = df_ibv.groupby('account_id')[['IBV_Credit','IBV_Debit']].sum().reset_index()
        f22.columns = ['account_id','TOTAL_CREDIT','TOTAL_DEBIT']
        
        # # Alert indicators added by Coral: stop payment, nsf, overdraft, return
        stop_keywords = ['e-transfer stop', 'stop payt','stop payment','stop fee','stop pmt','stoppa','stop pymt']
        nsf_keywords = [r"\bnsf", 'NON-SUFFICIENT FUNDS','Insufficient fund','NON SUFFICIENT','Returned Item Fee']
        overdraft_keywords = ['Overdraft', 'Over limit','OD FEE','OD PROTECTION','OD HANDL']
        return_keywords = ['returned', 'RETURN FEE','Return of','EFT.*Return','EFT.*Reversal','rtn.*eft']
        gambling_keywords = ['payper', 'paybilt',' gigadat', 'playnow', 'bet.*river','betmgm','betty.*gaming',
                            'bally.*bet', 'pointsbet', 'bet365', 'betway', 'betano', 'thescore.*bet', 'northstar.*bet',
                            'lotto', 'playalberta', 'playtime', 'casino', 'ilixium.*casin']
        
        df_ibv['STOP_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(stop_keywords), case=False)) & (df_ibv['IBV_Credit']>0), 'STOP_Incident'] = 1
        df_ibv['NSF_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(nsf_keywords), case=False)), 'NSF_Incident'] = 1
        df_ibv['OVERDRAFT_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(overdraft_keywords), case=False)), 'OVERDRAFT_Incident'] = 1
        df_ibv['RETURN_Incident'] = 0
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(return_keywords), case=False)), 'RETURN_Incident'] = 1
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'Gambling_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(gambling_keywords), case=False)), 'IBV_Debit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'EI_Incident'] = \
         df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'EI\sAE|\sEI$|AE/EI'), case=False)), 'IBV_Credit'].values
        df_ibv.loc[(df_ibv['IBV_Description'].str.contains("|".join(r'bree'), case=False)), 'BREE_Incident'] = 1
        
        
        f23 = df_ibv.groupby('account_id').agg(NUM_STOP_PMT = ('STOP_Incident','sum'),
                                               NUM_NSF = ('NSF_Incident','sum'),
                                               NUM_OD = ('OVERDRAFT_Incident','sum'),
                                               NUM_RETURN_PMT = ('RETURN_Incident','sum'),
                                               NUM_GAMBLING_PMT = ('Gambling_Incident', 'count'),
                                               AMT_GAMBLING_PMT = ('Gambling_Incident', 'sum'),
                                               NUM_EI_PMT = ('EI_Incident', 'count'),
                                               AMT_EI_PMT = ('EI_Incident', 'sum'),
                                               NUM_Bree = ('BREE_Incident', 'count')
                                              ).reset_index()

        
        ibv_modelinput = f1.copy()
        for i in [f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23]:
            ibv_modelinput = ibv_modelinput.merge(i, on='account_id',how='left')

        # ## Step 3: DataCleaning after feature generation
        stepname = '4 DataCleaning after Feature Gen'
        ibv_modelinput = ibv_modelinput.fillna(value=0).replace(np.inf, 100).replace(-np.inf, -100)

        # ## Step 4: IBV Model Scoring and IBVBand Assignment based on IBVScore
        ibv_features = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT',
                'NUM_EI_PMT', 'AMT_EI_PMT', 'NUM_Bree']

        # Load Model
        stepname = '5 Model Loading & Scoring'
#         clf_ibv = joblib.load(modelfilepath)

        # Get IBV Sre (output 1) make sure 1000- is removed --> higher score is already better, different from default model
#         ibv_modelinput['IBVScore'] = np.round(clf_ibv.predict_proba(ibv_modelinput[ibv_features])[1][:,1][0]*1000,0).astype(int)

        # Get IBV Band (output 2)
#         ibv_modelinput['IBVBand'] = np.where(ibv_modelinput['IBVScore']< 547,1, 
#                                              np.where(ibv_modelinput['IBVScore']< 719,2, 
#                                             np.where(ibv_modelinput['IBVScore'] < 820,3,4) ) )
#         IBVScore = int(ibv_modelinput.IBVScore.values[0])
#         IBVBand = int(ibv_modelinput.IBVBand.values[0])
        IBVScore, IBVBand = 0, 0
        mask = df_ibv['IBV_Description'].astype('string').str.contains(r'eft.*(?:reversal|return)', case=False, regex=True, na=False)
        if mask.sum() >= 6:
            pass #             return "{\"ModelScore\":"+str(1)+ ",\"IBVBand\":" + str(1)+"}"
        if len(df_ibv[ (df_ibv['isLoan'] == 0) & (df_ibv['IBV_Credit'] >= 500) ]) < 5:
            pass #             return "{\"ModelScore\":"+str(1)+ ",\"IBVBand\":" + str(2)+"}"
    except:

        try:
            bankaccountid = df_ibv['account_id'].values[0]
            ndbaccountid = json_temp['NDB']['accountnumber']
            logging.exception('AccountID: ' + str(bankaccountid) + '; NDBAccount: ' + str(ndbaccountid) + '; ErrorInStep: ' + stepname )
        except: # If plaid file received is empty: account_id will be 0, or modified to Other value 
            logging.exception('AccountID: ' + 'Not Available' + '; ErrorInStep: ' + stepname )
        finally:
            return "{\"ModelScore\":"+str(0)+ ",\"IBVBand\":" + str(0)+"}"

 
    return ibv_modelinput[ibv_features]
# "{\"ModelScore\":"+str(IBVScore)+ ",\"IBVBand\":" + str(IBVBand)+"}"
    

#### Check to see if model works as intended for one json input file

In [7]:
data_input_path = 'retroData/'+str('IBV_126164.json')
with open(data_input_path) as f:
    data = f.read()
x = ibvmodel(data,0)
x

,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,...,TOTAL_DEBIT,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree
0,90,1.185957,0.349861,216.08575,223.13475,0.0,-0.471256,-0.167663,6694.0425,136.25,...,25930.29,0,2,2,1,0,0.0,654,26776.17,654


#### Now expand the engineering model to who data repo, which includes all json files

In [10]:
from tqdm import tqdm
fullyworkinglist = []
workingnulllist = []
notworkinglist = []

df_fe = pd.DataFrame([], columns = ['MaxHistory', 'CREDIT_TO_DEBIT_RATIO_MEAN', 'CREDIT_TO_DEBIT_RATIO_SD',
                'DAILY_DEBIT_AMOUNT_MEAN', 'DAILY_INCOME_MEAN',
                'DAILY_INCOME_REGULAR_MEAN', 'DEBIT_AMOUNT_Z', 'DEBIT_COUNT_Z',
                'HIGHEST_PAY_DEPOSIT_MEAN', 'HIGHEST_PAY_FREQUENCY',
                'INCOME_SOURCES_COUNT', 'MONTH_INFLOW_MEAN', 'MONTH_INFLOW_SD',
                'MONTH_OUTFLOW_MEAN', 'MONTH_OUTFLOW_SD', 'MONTHS_WITH_FEES_RATE',
                'MONTHS_WITH_EMPLOYMENT_RATE', 'NO_ACTIVITY_RATE',
                'OD_AND_NSF_FEES_DAILY', 'RECURRENT_COUNT', 'RECURRING_RATE',
                'BALANCE_MEAN','BALANCE_SD', 'BALANCE_ABOVE_RATE','LAST_BALANCE_ABOVE',
                'AVG_BALANCE_ABOVE_7D', 'HIGHER_BALANCE_7D', 'NUM_LOAN_PMT', 'TOTAL_LOAN_PMT_AMT',
                'NUM_LOAN_ORIG', 'TOTAL_LOAN_ORIG_AMT','PAYROLL_AMOUNT', 'TOTAL_CREDIT','TOTAL_DEBIT',
                'NUM_STOP_PMT','NUM_NSF','NUM_OD','NUM_RETURN_PMT', 'NUM_GAMBLING_PMT' ,'AMT_GAMBLING_PMT'])
for i in tqdm(arr):
    try:
        data_input_path = 'retroData/'+str(i)
        with open(data_input_path) as f:
            data = f.read()
        x = ibvmodel(data,0)
        x['AppID'] = i[4:-5]
        df_fe = pd.concat([df_fe,x], axis=0)
        if x.MaxHistory.isnull()[0] == True:
            workingnulllist.append(i)
        elif x.MaxHistory.isnull()[0] == False:
            fullyworkinglist.append(i)
    except: 
        notworkinglist.append(i)
        pass

  0%|          | 0/9655 [00:00<?, ?it/s]/var/folders/b7/d1qg7gm9009f_cps7l193hch0000gn/T/ipykernel_82377/3654372071.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fe = pd.concat([df_fe,x], axis=0)
100%|██████████| 9655/9655 [13:19<00:00, 12.08it/s]


In [11]:
len(fullyworkinglist), len(workingnulllist), len(notworkinglist)

(9608, 0, 47)

In [14]:
df_fe.AppID = df_fe.AppID.astype(int)
df_fe.head()

,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,DEBIT_AMOUNT_Z,DEBIT_COUNT_Z,HIGHEST_PAY_DEPOSIT_MEAN,HIGHEST_PAY_FREQUENCY,...,NUM_STOP_PMT,NUM_NSF,NUM_OD,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,AppID
0,84,0.999479,0.001491,666.073778,665.776333,0.000000,-0.330991,1.000000,19973.290000,46.333333,...,0,0,0,1,72,6925.0,205.0,59919.87,199.0,116891
0,90,1.040395,0.181353,380.691667,379.859667,0.000000,-0.822891,-0.907459,11395.790000,113.500000,...,0,11,2,6,2,25.0,535.0,45583.16,482.0,65258
0,88,0.863058,0.295284,109.585833,89.029083,0.000000,-1.293699,-1.218667,1331.875000,7.000000,...,0,2,0,1,0,0.0,468.0,15140.15,460.0,44155
0,64,0.783197,0.418876,91.086250,92.264750,84.447444,-1.158455,-0.968014,2533.423333,2.000000,...,0,0,0,0,0,0.0,154.0,11071.77,119.0,2152
0,89,1.142887,0.240170,241.105778,257.049333,0.000000,1.030238,0.458831,6888.930000,20.666667,...,0,0,0,0,0,0.0,191.0,32514.70,191.0,48875


In [24]:
from sqlalchemy import create_engine 
import urllib
server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER
-- From Anson
drop table if EXISTS #t1
select A.Application_ID, A.PortfolioID, A.CustomerID, A.ApplicationDate,
case when ApplicationSteps like '%S%' then 1 else 0 end as NewlyScored,
case when A.ApplicationStatus in ('A','P') then 1 else 0 end as Accepted,
case when A.ApplicationStatus in ('A','P') then A.LeadPurchasePrice else 0 end as LeadPurchasePrice,
case when L.LoanStatus not in ('V','W','G','K') then 1 else 0 end as Originated,
L.LoanID, datediff(year, VW.DOB, A.ApplicationDate) as Age,
case when VW.Frequency in ('B','S') then 'B' else VW.Frequency end as Frequency,
case when L.LoanStatus not in ('V','W','G','K') then L.OriginatedAmount else null end as OriginatedAmount,
LP.Provider_name 
into #t1
from Application A
left join Loans L on A.Application_ID = L.ApplicationID and A.PortFolioID = L.PortFolioID
left join LeadProvider LP on A.LeadProviderID = LP.LeadProviderID
left join LF_LMS_Logs..VW_ApplicationDump VW on A.APPGUID = VW.APPGUID
where A.ApplicationDate>'2023-04-30' and datediff(week, A.ApplicationDate, getdate()) >2 
-- and A.ApplicationSteps not like '%R%' and A.ApplicationSteps not like '%O%' 
order by A.ApplicationDate asc


-- scoring result for PlaidUDW_v1 and NegativeDBModelLP_v1
drop table if EXISTS #t2
select *
into #t2 from
(select A.*, 
cast(substring(B1.ExtResponse, CHARINDEX('IBVBand',B1.ExtResponse)+9,1) as int) as NDBand,
cast(substring(B2.ExtResponse, CHARINDEX('NDBand',B2.ExtResponse)+8,1) as int) as IBVBand,
row_number() over (partition by A.Application_ID, A.PortfolioID order by B.ApplicationDate desc) as RN
from #t1 A 
left join Application B on A.CustomerID = B.CustomerID and A.ApplicationDate >= B.ApplicationDate and B.ApplicationStatus like '%S%'
left join ScoringPythonResult B1 on B.Application_ID = B1.ApplicationID and B.PortFolioID = B1.PortfolioID and B1.iLabel= 'IBVBand' 
left join ScoringPythonResult B2 on B.Application_ID = B2.ApplicationID and B.PortFolioID = B2.PortfolioID and B2.iLabel= 'NDBand' ) K where RN=1



ALTER TABLE #t2
DROP COLUMN RN;


DROP TABLE IF EXISTS #LF_LoanDefault
SELECT L.LoanID, L.ApplicationID AS Application_ID, A.APPGUID, A.ApplicationDate, A.ApplicationSteps, L.PortFolioID, L.LoanStatus,
P.InstallmentNumber, P.PaymentStatus, P.PaymentType, P.PaymentMode, P.AttemptNo, P.TransactionDate, P.PaymentID,
I.InstallmentID, I.iPaymentMode, I.DueDate, I.Status, -- used to exclude pendings (code 684)
(CASE WHEN I.Status=684 THEN 1 ELSE 0 END) AS Pending
INTO #LF_LoanDefault
FROM LF_LMSMaster..Loans L
LEFT JOIN LF_LMSMaster..Payment P ON P.LoanID = L.LoanID
LEFT JOIN LF_LMSMaster..Installments I ON I.InstallmentID = P.InstallmentID
LEFT JOIN LF_LMSMaster..Application A ON A.PortfolioID=L.PortfolioID AND A.Application_ID = L.ApplicationID
WHERE A.ApplicationDate>'2023-04-30' AND datediff(week, A.ApplicationDate, getdate()) >2 
AND I.InstallmentNumber = 1

DROP TABLE IF EXISTS #LoanDefault_Flag
SELECT 
    L.*,
    -- FPDFA flag
    CASE 
        WHEN L.PaymentStatus = 'R'
             AND L.PaymentType IN ('I','S','A')
             AND L.PaymentMode IN ('A','B','D')
             AND L.DueDate <= CAST(GETDATE() AS date)
             AND NOT EXISTS (
                 SELECT 1
                 FROM #LF_LoanDefault ld
                 WHERE ld.InstallmentID = L.InstallmentID
                   AND ld.PaymentStatus = 'D'
                   AND ld.PaymentType NOT IN ('3','~','Q')
                   AND ld.PaymentMode IN ('A','D','B')
                   AND CONVERT(date, ld.TransactionDate) = CONVERT(date, L.DueDate)
             )
        THEN 1 ELSE 0 END AS is_FPDFA,
        CASE 
        WHEN L.LoanStatus NOT IN ('V','W','G','K')
             AND NOT (
                 L.iPaymentMode = 144 
                 AND L.Pending = 1
                 AND L.DueDate >= CAST(GETDATE() AS date)
             )
        THEN 1 ELSE 0 
    END AS is_loan_first_install
INTO #LoanDefault_Flag
FROM #LF_LoanDefault L;


DROP TABLE IF EXISTS #LoanDefault_Dedup;
WITH dedup AS (
    SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
           InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
           PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install,
           ROW_NUMBER() OVER (PARTITION BY Application_ID, PortfolioID ORDER BY is_FPDFA DESC) AS rn
    FROM #LoanDefault_Flag
)
SELECT LoanID, Application_ID, APPGUID, ApplicationDate, ApplicationSteps, PortfolioID, LoanStatus,
       InstallmentNumber, PaymentStatus, PaymentType, PaymentMode, AttemptNo, TransactionDate,
       PaymentID, InstallmentID, iPaymentMode, DueDate, Status, Pending, is_FPDFA, is_loan_first_install
INTO #LoanDefault_Dedup
FROM dedup
WHERE rn = 1;
-- Check Top 100
SELECT TOP 10 *
FROM #LoanDefault_Dedup


drop table if EXISTS #t3
select A.*, B.is_FPDFA, B.PaymentStatus, B.LoanStatus
into #t3
from #t2 A 
inner join #LoanDefault_Dedup B on A.LoanID = B.LoanID


drop table if EXISTS #t31
select Application_ID, PortFolioID, LoanID, (1-max(FstInstPaidOff)) as FPDAA
into #t31 from
(select A.*, case when P.PaymentStatus = 'D' then 1 else 0 end as FstInstPaidOff, P.PaymentStatus
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode IN ('A','D','B') and PaymentType NOT IN ('3','~','Q')
                        and P.InstallmentNumber=1 and P.AttemptNo >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R','S','B')
where A.Originated=1) K group by Application_ID, PortFolioID, LoanID


drop table if EXISTS #t4
select A.Application_ID, A.PortFolioID, A.LoanID, sum(case when P.PaymentStatus = 'D' then P.PaymentAmount else 0 end) as PaidOffPaymentAmount,
sum(case when P.PaymentStatus = 'R' then 1 else 0 end) as PmtReturn, count(*) as PmtCount
into #t4
from #t2 A 
inner join Payment P on A.LoanID = P.LoanID and P.PaymentMode not in ('V','P','T','H','G') and PaymentType in ('I','S','Z','A','X','Q') 
                        and P.InstallmentNumber >= 1 and P.PaymentDate <= getdate() and P.PaymentStatus in ('D','R') and P.PaymentAmount>=2.95 and (P.PaymentAmount > P.FeeChargePaid)
where A.Originated=1
group by A.Application_ID, A.PortFolioID, A.LoanID
"""
query2 = """
select A.*,B.is_FPDFA, B2.FPDAA, C.PaidOffPaymentAmount, C.PmtCount, C.PmtReturn,
C.PaidOffPaymentAmount / (A.OriginatedAmount+0.0) as Payin,
case when C.PaidOffPaymentAmount >= A.OriginatedAmount * 1.05 then 1 else 0 end as IsGood
from #t2 A  -- avg(OriginatedAmount+0.0) as AvgLoanAmount, sum(FPDFA) as FPDFA, sum(FPDFA+0.0)/count(*)*100 as FPDFARate
left join #t3 B on A.Application_ID = B.Application_ID and A.PortFolioID = B.PortFolioID
left join #t31 B2 on A.Application_ID = B2.Application_ID and A.PortFolioID = B2.PortFolioID
left join #t4 C on A.Application_ID = C.Application_ID and A.PortFolioID = C.PortFolioID
where A.Originated = 1 and ((datediff(day, A.ApplicationDate, getdate()) >= 110) or (C.PaidOffPaymentAmount > A.OriginatedAmount))
"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf = df_perf.rename(columns = {'Application_ID':'AppID'})
df_perf.AppID = df_perf.AppID.astype(int)
df_perf.head(3)

,AppID,PortfolioID,CustomerID,ApplicationDate,NewlyScored,Accepted,LeadPurchasePrice,Originated,LoanID,Age,...,Provider_name,NDBand,IBVBand,is_FPDFA,FPDAA,PaidOffPaymentAmount,PmtCount,PmtReturn,Payin,IsGood
0,172,1,132,2023-05-01 09:48:10,1,1,5.0,1,I49-0,35.0,...,Dot818,None,None,0.0,0.0,69.2,6.0,5.0,0.2768,0
1,174,1,134,2023-05-01 10:14:45,1,1,5.0,1,I50-0,51.0,...,Dot818,None,None,0.0,0.0,69.2,6.0,5.0,0.2768,0
2,197,1,153,2023-05-01 14:08:01,1,1,5.0,1,I60-0,27.0,...,Dot818,None,None,1.0,1.0,346.0,7.0,2.0,1.3840,1


In [26]:
# For FPD Modeling
df_final = df_perf[['AppID','CustomerID','ApplicationDate','Age']].merge(df_fe, on=['AppID'],how='inner')
# df_final = df_final.rename(columns = {'AppID':'ScoredAppID','ApplicationDate':"ScoredAppDate"})
df_final = df_final.merge(df_perf[df_perf.FPDAA.notnull()][['IsGood','AppID','Payin','FPDAA','is_FPDFA']], how='inner', on=['AppID'])
# len(df_final[(df_final.FPDAA.notnull())&(df_final.MaxHistory.notnull())]) # 1k makes sense

In [27]:
df_final[(df_final.IsGood.notnull())].to_csv('ibv_input_data.csv',index=False)

In [31]:
df_final.head()

,AppID,CustomerID,ApplicationDate,Age,MaxHistory,CREDIT_TO_DEBIT_RATIO_MEAN,CREDIT_TO_DEBIT_RATIO_SD,DAILY_DEBIT_AMOUNT_MEAN,DAILY_INCOME_MEAN,DAILY_INCOME_REGULAR_MEAN,...,NUM_RETURN_PMT,NUM_GAMBLING_PMT,AMT_GAMBLING_PMT,NUM_EI_PMT,AMT_EI_PMT,NUM_Bree,IsGood,Payin,FPDAA,is_FPDFA
0,172,132,2023-05-01 09:48:10,35.0,63,1.023761,0.866899,225.379417,225.313083,189.683333,...,0,0,0.0,305.0,30387.57,287.0,0,0.2768,0.0,0.0
1,174,134,2023-05-01 10:14:45,51.0,90,1.194366,1.061650,195.728267,205.952200,124.399500,...,0,0,0.0,567.0,30892.83,458.0,0,0.2768,0.0,0.0
2,197,153,2023-05-01 14:08:01,27.0,60,1.027346,0.047099,239.111111,239.341778,95.763167,...,0,0,0.0,236.0,21540.76,221.0,1,1.3840,1.0,1.0
3,308,244,2023-05-04 10:52:57,39.0,90,1.100375,0.236295,230.337083,256.043333,156.048250,...,0,0,0.0,520.0,30725.47,500.0,0,0.9688,0.0,0.0
4,340,273,2023-05-04 14:48:10,26.0,88,0.832004,0.365121,104.749083,104.939917,9.995444,...,3,61,466.0,242.0,12592.79,224.0,0,0.6559,0.0,0.0


In [28]:
print(len(df_final[(df_final.IsGood.notnull())&(df_final.MaxHistory.isnull())])) # without model step 1
print(len(df_final[(df_final.IsGood.notnull())&(df_final.MaxHistory.notnull())])) # with model step 2

0
7968
